In [71]:
import tensorflow as tf
import pandas as pd
import numpy as np


In [72]:
columns_name = ['Sepallength', 'Sepalwidth', 'Petallength', 'Petalwidth', 'label']
data = pd.read_csv('iris_training.csv', sep=",")
data.columns = columns_name

In [73]:
col_map = {"sepal.length": 'Sepallength',
         "sepal.width": 'Sepalwidth',
         "petal.length": 'Petallength',
         "petal.width": 'Petalwidth',
         "variety": 'label'}

data = data.rename(index=str, columns=col_map)

data[:5]

,Sepallength,Sepalwidth,Petallength,Petalwidth,label
0,6.4,2.8,5.6,2.2,2
1,5.0,2.3,3.3,1.0,1
2,4.9,2.5,4.5,1.7,2
3,4.9,3.1,1.5,0.1,0
4,5.7,3.8,1.7,0.3,0


In [74]:
x_train, y_train = data.iloc[:, 0:4], data.iloc[:, -1:]
features = {key: np.array(value) for key, value in dict(x_train).items()}
target = {key: np.array(value) for key,value in dict(y_train).items()}

print(target)

{'label': array([2, 1, 2, 0, 0, 0, 0, 2, 1, 0, 1, 1, 0, 0, 2, 1, 2, 2, 2, 0, 2, 2,
       0, 2, 2, 0, 1, 2, 1, 1, 1, 1, 1, 2, 2, 2, 2, 2, 0, 0, 2, 2, 2, 0,
       0, 2, 0, 2, 0, 2, 0, 1, 1, 0, 1, 2, 2, 2, 2, 1, 1, 2, 2, 2, 1, 2,
       0, 2, 2, 0, 0, 1, 0, 2, 2, 0, 1, 1, 1, 2, 0, 1, 1, 1, 2, 0, 1, 1,
       1, 0, 2, 1, 0, 0, 2, 0, 0, 2, 1, 0, 0, 1, 0, 1, 0, 0, 0, 0, 1, 0,
       2, 1, 0, 2, 0, 1, 1, 0, 0, 1])}


# 1. input_layer 层

```python

tf.feature_column.input_layer(
    features,
    feature_columns,
    weight_collections=None,
    trainable=True,
    cols_to_vars=None,
    cols_to_output_tensors=None
)
```

使用 input_layer 作为 model 的一个 input layer。

+ features: 字典，最主要的是 dict的key一定要与 feature_columns的key一致，后续才能 才能根据key进行匹配。
+ feature_columns：改参数必须是 继承于DenseColumn的 numeric_column, embedding_column, bucketized_column, indicator_column。如果feature是类别的，那么必须先用embedding_column or indicator_column封装一下使用。

# 2. Numeric feature columns

## 2.1 numeric_column

```python
tf.feature_column.numeric_column(
    key,
    shape=(1,),
    default_value=None,
    dtype=tf.dtypes.float32,
    normalizer_fn=None
)
```

参数说明：

+ key: 唯一确定输入特征的字符串，也就是说该字段用来标识这个特征。
+ shape：该特征的形状。
+ default_value 默认值。
+ dtype：数据类型。
+ normalizer_fn：做一些normalization操作，比如标准化。normalizer 函数 input: tensor；output:tensor。

In [76]:
feature_columns = [tf.feature_column.numeric_column(key, default_value=0) for key in x_train.keys()]

feature_columns

[_NumericColumn(key='Sepallength', shape=(1,), default_value=(0,), dtype=tf.float32, normalizer_fn=None),
 _NumericColumn(key='Sepalwidth', shape=(1,), default_value=(0,), dtype=tf.float32, normalizer_fn=None),
 _NumericColumn(key='Petallength', shape=(1,), default_value=(0,), dtype=tf.float32, normalizer_fn=None),
 _NumericColumn(key='Petalwidth', shape=(1,), default_value=(0,), dtype=tf.float32, normalizer_fn=None)]

In [77]:
# 使用 input_layer 作为 model的一个 input layer

inn = tf.feature_column.input_layer(features, feature_columns)
print(inn)

with tf.Session() as sess:
    print(sess.run(inn[:5]))

Tensor("input_layer_22/concat:0", shape=(120, 4), dtype=float32)
[[5.6 2.2 6.4 2.8]
 [3.3 1.  5.  2.3]
 [4.5 1.7 4.9 2.5]
 [1.5 0.1 4.9 3.1]
 [1.7 0.3 5.7 3.8]]


In [78]:
# 使用 normalizer_fn, 标准化

mean = x_train['Sepallength'].mean()
std = x_train['Sepallength'].std()
print('mean is {}, std is {}'.format(mean, std))

def normalization(mean, std):
    def normalizer_fn(x):
        return (x-mean) / std   
    return normalizer_fn

mean is 5.845, std is 0.8685784774150068


In [79]:
feature = tf.feature_column.numeric_column('Sepallength', 
                                           default_value=0, 
                                           normalizer_fn=normalization(mean, std))  

print(feature)
print("========================\n")

in_Sepallength = tf.feature_column.input_layer(features, [feature])
print(in_Sepallength)
print("========================\n")

with tf.Session() as sess:
    print(sess.run(in_Sepallength[:5]))

_NumericColumn(key='Sepallength', shape=(1,), default_value=(0,), dtype=tf.float32, normalizer_fn=<function normalization.<locals>.normalizer_fn at 0x1520ec1bf950>)

Tensor("input_layer_23/concat:0", shape=(120, 1), dtype=float32)

[[ 0.6389751 ]
 [-0.97285396]
 [-1.0879846 ]
 [-1.0879846 ]
 [-0.16693944]]


# 3. categorical feature columns

## 3.1 tf.feature_column.categorical_column_with_identity

可以理解为将类型为int的特征，转换为one-hot类型的dense tensor.

```python
tf.feature_column.categorical_column_with_identity(
    key,
    num_buckets,
    default_value=None
)
```

首先它返回的是唯一值的类别 column (categorical column)

+ key：参数同上。
+ num_buckets: 限制输入输出的数据范围。范围是[0, num_buckets)，还有就是使用这个函数，对应的feature value必须是int类型。
+ default_value: 默认值，该值必须在 [0, num_buckets)范围内。如果不设置默认值，则会抛出异常（如果数据不在[0, num_buckets)范围内）。

In [80]:
features_new  = {'id': [0, 1, 2, 3, 4, 4, 6, 7, 8, 9],
                 'cat': ['test', 'train', 'eval', 'train', 'train', 
                                  'test', 'eval', 'eval', 'test', 'train']}

print(features_new)

{'id': [0, 1, 2, 3, 4, 4, 6, 7, 8, 9], 'cat': ['test', 'train', 'eval', 'train', 'train', 'test', 'eval', 'eval', 'test', 'train']}


In [81]:
ci = tf.feature_column.categorical_column_with_identity('id', 10)
ci_int = tf.feature_column.indicator_column(ci)

# 为什么添加这个步骤呢？因为 input_layer的入参是有要求的，如上所说必须是继承于DenseColumn的类型,详看下面的解释

In [82]:
in_ci = tf.feature_column.input_layer(features_new, [ci_int])

with tf.Session() as sess:
    print(sess.run(in_ci))

[[1. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 1. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 1. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 1. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 1. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 1. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 1. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 1. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 1. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 1.]]


## 3.2 tf.feature_column.indicator_column

tf.feature_column.indicator_column(categorical_column)

+ 参数：必须是categorical_column，如上可知，只有categorical_column_with*, crossed_column以及bucketized_column 类型的column才可以使用该函数。
+ 作用：将categorical_column表示成 multi-hot形式的 dense tensor。

下面官方还特意提到了一点就是：indicator_column 可以将任意的 categorical_column_with* 作为参数（同我上面说的一致），但是，如果说 buckets/unique（分箱，或者 ont-hot）的值比较大的时候，还是建议使用embedding_column。

## 3.3 tf.feature_column.categorical_column_with_vocabulary_file

```python
tf.feature_column.categorical_column_with_vocabulary_file( key,
    vocabulary_file,
    vocabulary_size=None,
    num_oov_buckets=0,
    default_value=None,
    dtype=tf.dtypes.string
    )
```

+ 作用：将表示类别的feature转换为Categorical Column tensor。
+ 参数：
    + key: 同上。
    + vocabulary_file：字典文件。
    + vocabulary_size：字典文件中元素个数，该值不能大于字典元素数；如果该值小于字典元素数，则字典中后续的元素将被忽略。默认情况下，该值会被自动设置为字典元素数。
    + num_oov_buckets：就是说如果value值超出了字典，后续应该用什么值来填充，基于hash算法，填充值在 [vocabulary_size, vocabulary_size + num_oov_buckets)范围内。注意该值与 default_value 不共存(不能同时存在，否则会报错)。
    + default_value：默认-1，就是说当 feature value的值不在字典中时，也就是oov，使用一个整数默认值来填充。
    + dtype：feature type，目前仅支持 string 和 integer。

```shell
echo "train" >> vocabulary_file.csv
echo "test" >> vocabulary_file.csv
echo "eval" >> vocabulary_file.csv
```

In [83]:
cate_vf = tf.feature_column.categorical_column_with_vocabulary_file('cat',
                vocabulary_file='vocabulary_file.csv',
                vocabulary_size=2,
                num_oov_buckets=3)

ind_cat = tf.feature_column.indicator_column(cate_vf)
print(ind_cat)

inp_cat = tf.feature_column.input_layer(features_new, [ind_cat])
print(inp_cat)

with tf.Session() as sess:
    # 此处必须使用 tf.tables_initializer来初始化 lookuptable
    sess.run(tf.tables_initializer())
    print(sess.run(inp_cat))

_IndicatorColumn(categorical_column=_VocabularyFileCategoricalColumn(key='cat', vocabulary_file='vocabulary_file.csv', vocabulary_size=2, num_oov_buckets=3, dtype=tf.string, default_value=-1))
Tensor("input_layer_25/concat:0", shape=(10, 5), dtype=float32)
[[0. 1. 0. 0. 0.]
 [1. 0. 0. 0. 0.]
 [0. 0. 0. 0. 1.]
 [1. 0. 0. 0. 0.]
 [1. 0. 0. 0. 0.]
 [0. 1. 0. 0. 0.]
 [0. 0. 0. 0. 1.]
 [0. 0. 0. 0. 1.]
 [0. 1. 0. 0. 0.]
 [1. 0. 0. 0. 0.]]


## 3.4 tf.feature_column.categorical_column_with_vocabulary_list

```python
tf.feature_column.categorical_column_with_vocabulary_list(key,
    vocabulary_list,
    dtype = None,
    default_value = -1,
    num_oov_buckets = 0
    )
```

+ 除了vocabulary_list参数外，其它都与categorical_column_with_vocabulary_file一致。
+ vocabulary_list：指定字典列表，注意顺序，解析后的数据是按照 list 索引决定的。

In [84]:
cate_vl = tf.feature_column.categorical_column_with_vocabulary_list('cat',
                vocabulary_list = ['test', 'train', 'eval'],
                default_value = 1)

print(cate_vl)

ind_cat_l = tf.feature_column.indicator_column(cate_vl)
inp_cat_l = tf.feature_column.input_layer(features_new, [ind_cat_l])

with tf.Session() as sess:
    # 在此处必须使用 tf.tables_initializer来初始化 lookuptable
    sess.run(tf.tables_initializer())
    print(sess.run(inp_cat_l))

_VocabularyListCategoricalColumn(key='cat', vocabulary_list=('test', 'train', 'eval'), dtype=tf.string, default_value=1, num_oov_buckets=0)
[[1. 0. 0.]
 [0. 1. 0.]
 [0. 0. 1.]
 [0. 1. 0.]
 [0. 1. 0.]
 [1. 0. 0.]
 [0. 0. 1.]
 [0. 0. 1.]
 [1. 0. 0.]
 [0. 1. 0.]]


In [86]:
features_multi = {'id': [0, 1, 2, 3, 4, 4, 6, 7, 8, 9], 
                  'cat': ['test train', 'train eval', 'eval', 'train', 
                          'train', 'test', 'eval', 'eval', 'test', 'train']}

cate_multi = tf.feature_column.categorical_column_with_vocabulary_list('cat',
                vocabulary_list = ['test', 'train', 'eval'],
                default_value = 2)

print(cate_multi)
print("\n")

ind_multi=tf.feature_column.indicator_column(cate_multi)
inp_multi=tf.feature_column.input_layer(features_multi,[ind_multi])

with tf.Session() as sess:
    # 在此处必须使用 tf.tables_initializer来初始化 lookuptable
    sess.run(tf.tables_initializer())
    print(sess.run(inp_multi))

_VocabularyListCategoricalColumn(key='cat', vocabulary_list=('test', 'train', 'eval'), dtype=tf.string, default_value=2, num_oov_buckets=0)


[[0. 0. 1.]
 [0. 0. 1.]
 [0. 0. 1.]
 [0. 1. 0.]
 [0. 1. 0.]
 [1. 0. 0.]
 [0. 0. 1.]
 [0. 0. 1.]
 [1. 0. 0.]
 [0. 1. 0.]]


## 3.5 tf.feature_column.categorical_column_with_hash_bucket

```python
tf.feature_column.categorical_column_with_hash_bucket(
    key,
    hash_bucket_size,
    dtype=tf.dtypes.string
    )
```

+ 作用：利用hash算法，将特征值类型为 integer/string 的特征进行分箱操作。
+ 参数：
    + key：同上。
    + hash_bucket_size：int类型且>1。
    + dtype：特征的类型，目前仅支持 string 和integer。

In [87]:
cate_hb = tf.feature_column.categorical_column_with_hash_bucket('id',
                hash_bucket_size = 5,
                dtype = tf.int64)

ind_cat_hb = tf.feature_column.indicator_column(cate_hb)
inp_cat_hb = tf.feature_column.input_layer(features_new, [ind_cat_hb])

with tf.Session() as sess:
    print(sess.run(inp_cat_hb))

[[1. 0. 0. 0. 0.]
 [0. 0. 0. 0. 1.]
 [0. 0. 0. 0. 1.]
 [0. 1. 0. 0. 0.]
 [0. 0. 0. 1. 0.]
 [0. 0. 0. 1. 0.]
 [0. 0. 1. 0. 0.]
 [0. 0. 0. 1. 0.]
 [0. 0. 1. 0. 0.]
 [1. 0. 0. 0. 0.]]


## 3.6 tf.feature_column.bucketized_column

```python
tf.feature_column.bucketized_column(
    source_column,
    boundaries
    )
```

+ 作用：分箱目的是离散化数据，比如将年龄划分为：少年，青年，壮年以及老年。
+ 参数：
    + source_column：numeric_column。
    + boundaries：边界列表，形成 左闭右开 区间，比如 [1,2,3]，那么拆分后的边界将是 (-inf,1), [1,2), [2,3), [3,+inf) 四个区间。

In [90]:
bc_fc = tf.feature_column.numeric_column('id', default_value = 0)
bucket = tf.feature_column.bucketized_column(bc_fc, boundaries = [3, 5, 8])

print(bucket)

input_bucket=tf.feature_column.input_layer(features_new, [bucket])

with tf.Session() as sess:
    print("\n**********")
    print(input_bucket)
    print("**********\n")
    print(sess.run(input_bucket))

_BucketizedColumn(source_column=_NumericColumn(key='id', shape=(1,), default_value=(0,), dtype=tf.float32, normalizer_fn=None), boundaries=(3, 5, 8))

**********
Tensor("input_layer_32/concat:0", shape=(10, 4), dtype=float32)
**********

[[1. 0. 0. 0.]
 [1. 0. 0. 0.]
 [1. 0. 0. 0.]
 [0. 1. 0. 0.]
 [0. 1. 0. 0.]
 [0. 1. 0. 0.]
 [0. 0. 1. 0.]
 [0. 0. 1. 0.]
 [0. 0. 0. 1.]
 [0. 0. 0. 1.]]


## 3.7 插一个方法 make_parse_example_spec

如果之前做过 tf 的serving，那么很有可能使用到 tf.FixedLenFeature / tf.VarLenFeature 这类方法，定义时，需要指定shape。
如果对这块理解不到位，那么shape很容易写错，导致 export model 失败。但是现在有了 make_parse_example_spec 方法，可以方便的为您进行转换。

如下所示：

In [91]:
tt = tf.feature_column.make_parse_example_spec([bucket])

print(tt)

#with tf.Session() as sess:
#    print(sess.run(tt))

{'id': FixedLenFeature(shape=(1,), dtype=tf.float32, default_value=(0,))}


## 3.8 tf.feature_column.crossed_column

```python
tf.feature_column.crossed_column(
    keys,
    hash_bucket_size,
    hash_key=None
    )
```

+ 说明：该函数的作用主要是通过笛卡尔积产生新的特征，然后经过hash算法产生最终特征：Hash(特征的笛卡尔积) % hash_bucket_size。
+ 参数：
    + keys：指明哪些 feature 需要进行合并形成新的特征，注意，该值的类型可以是 string(也就是数据集的column)，也可以是经过变换后的 categoricalColumn，但是不支持 hashed categorical。
    + hash_bucket_size：分箱的长度，可以理解为合并后数据的列数。

In [92]:
id_x_cat = tf.feature_column.crossed_column(['id', 'cat'], 5)

fc_ic = tf.feature_column.indicator_column(id_x_cat)
inp_cc = tf.feature_column.input_layer(features_new, [fc_ic])

print(inp_cc)
print("\n")

with tf.Session() as sess:
    print(sess.run(inp_cc))

Tensor("input_layer_33/concat:0", shape=(?, 5), dtype=float32)


[[0. 1. 0. 0. 0.]
 [0. 1. 0. 0. 0.]
 [0. 0. 1. 0. 0.]
 [0. 0. 0. 0. 1.]
 [0. 0. 1. 0. 0.]
 [1. 0. 0. 0. 0.]
 [0. 0. 0. 1. 0.]
 [0. 0. 0. 0. 1.]
 [0. 0. 0. 0. 1.]
 [0. 1. 0. 0. 0.]]


In [93]:
# 关于合并后的特征，serving时，可以看下效果

features = tf.feature_column.make_parse_example_spec([id_x_cat])
print(features)

{'id': VarLenFeature(dtype=tf.string), 'cat': VarLenFeature(dtype=tf.string)}


## 3.9 tf.feature_column.embedding_column

```python

tf.feature_column.embedding_column(
    categorical_column,
    dimension,
    combiner='mean',
    initializer=None,
    ckpt_to_load_from=None,
    tensor_name_in_ckpt=None,
    max_norm=None,
    trainable=True
    )
```

+ 作用：该方法和indicator_column一样，只接受 categorical_column，目的是将稀疏矩阵转换为稠密矩阵，word2vec。
+ 参数：
    + categorical_column：入参categorical_column_with* 的返回。
    + dimension：embedding 的维度，一般计算规则是类别开4次方，但是也可以根据需要自行设置。
    + combiner：多个vector的组合方式，有 mean(default), sqrtn以及sum。
    + initializer：embedding matrix 的初始化值，默认均值0，标准差 1/sqrt(dimension)的tf.truncated_normal_initializer。
    + ckpt_to_load_from以及tensor_name_in_ckpt：主要是为了使用pre-trained embedding matrix。
    + max_norm：if not 'None'，则使用l2归一化。
    + trainable：是否可训练的。

In [94]:
print(features_new)
print("\n")

cate_eb = tf.feature_column.categorical_column_with_vocabulary_list('cat',
                vocabulary_list=['test', 'train', 'eval'],                                                                 
                default_value=1)

print(cate_eb)

{'id': [0, 1, 2, 3, 4, 4, 6, 7, 8, 9], 'cat': ['test', 'train', 'eval', 'train', 'train', 'test', 'eval', 'eval', 'test', 'train']}


_VocabularyListCategoricalColumn(key='cat', vocabulary_list=('test', 'train', 'eval'), dtype=tf.string, default_value=1, num_oov_buckets=0)


In [96]:
eb_col = tf.feature_column.embedding_column(cate_eb, 3)

print(eb_col)
print()

inp_eb=tf.feature_column.input_layer(features_multi,[eb_col])

with tf.Session() as sess:
    # 在此处必须使用 tf.tables_initializer来初始化 lookuptable
    sess.run([tf.global_variables_initializer(), tf.tables_initializer()])
    print(sess.run(inp_eb))
    print()
    print(inp_eb.shape)

_EmbeddingColumn(categorical_column=_VocabularyListCategoricalColumn(key='cat', vocabulary_list=('test', 'train', 'eval'), dtype=tf.string, default_value=1, num_oov_buckets=0), dimension=3, combiner='mean', initializer=<tensorflow.python.ops.init_ops.TruncatedNormal object at 0x151f5dbe5cf8>, ckpt_to_load_from=None, tensor_name_in_ckpt=None, max_norm=None, trainable=True)

[[-0.78481716  0.6328265   0.6165252 ]
 [-0.78481716  0.6328265   0.6165252 ]
 [-0.14772381 -0.2884772  -0.597014  ]
 [-0.78481716  0.6328265   0.6165252 ]
 [-0.78481716  0.6328265   0.6165252 ]
 [ 0.5468127   1.0355408   0.01168238]
 [-0.14772381 -0.2884772  -0.597014  ]
 [-0.14772381 -0.2884772  -0.597014  ]
 [ 0.5468127   1.0355408   0.01168238]
 [-0.78481716  0.6328265   0.6165252 ]]

(?, 3)


# 参考

+ [https://github.com/AlbertBJ/tensorflow-summary/blob/master/feature_column/feature_column.ipynb](https://github.com/AlbertBJ/tensorflow-summary/blob/master/feature_column/feature_column.ipynb)

+ [https://github.com/tensorflow/docs/blob/master/site/en/r1/guide/feature_columns.md](https://github.com/tensorflow/docs/blob/master/site/en/r1/guide/feature_columns.md)